In [3]:
import cv2 
import mediapipe as mp

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.download_utils
pose = mp_pose.Pose()

# Start capturing video
cap = cv2.VideoCapture(0) # Use 0 for webcam, or provide a video file path

# Get video width, height, and FPS for saving
frame_width = int(cap.get(3))
frame_height = int(cap.get(3))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define video writer to save the output
out = cv2.VideoWriter('pose_output.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

while cap.isOpened() :
    
    ret,frame = cap.read()

    if not ret:
        break

    # Convert frame to RGB (MediaPipe requires RGB format)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)    

    #get Pose landmarks
    results = pose.process(rgb_frame)


    # Draw pose Landmarks
    if results.pose_landmarks:
        # mp_drawing.draw_landmarks(frame,results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        #get shoulder coordinates
        left_shoulder = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER]
        right_shoulder = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER]

        # Convert normalized coordinates (0-1) to pixel values
        left_shoulder_x,left_shoulder_y = int(left_shoulder.x * frame_width) , int(left_shoulder.y * frame_height)
        right_shoulder_x,right_shoulder_y = int(right_shoulder.x * frame_width) , int(right_shoulder.y * frame_height)
        
        # print coordinates
        print(f"Left Shoulder: ({left_shoulder_x}, {left_shoulder_y}), Right Shoulder: ({right_shoulder_x}, {right_shoulder_y})")

        # Draw Circles on Shoulders
        cv2.circle(frame,(left_shoulder_x,left_shoulder_y), 5,(0,255,0),-1)
        cv2.circle(frame,(right_shoulder_x,right_shoulder_y), 5,(0,255,0),-1)

    # Save video Frame
    out.writer(frame)

    #show output
    cv2.imshow("Pose Detection",frame)


    #press q tp quit

    if cv2.waitkey(1) & 0xff == ord(q):
        break


cap.release()
out.release()
cv2.destroyAllWindows()

